In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Define class names
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

In [2]:
# Import libraries
import torch
import torchvision.transforms as transforms
from torchvision import models

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])# Define transforms: resize, convert to tensor, normalize

In [3]:
import torchvision.transforms.functional as F

def unnormalize(img_tensor):
    device = img_tensor.device
    if img_tensor.dim() == 4:
        mean = torch.tensor([0.485, 0.456, 0.406], device=device).view(1, 3, 1, 1)
        std = torch.tensor([0.229, 0.224, 0.225], device=device).view(1, 3, 1, 1)
    else:
        mean = torch.tensor([0.485, 0.456, 0.406], device=device).view(3, 1, 1)
        std = torch.tensor([0.229, 0.224, 0.225], device=device).view(3, 1, 1)
    return img_tensor * std + mean

In [4]:
import torch.nn as nn
from torchvision.models import GoogLeNet_Weights

# Load GoogleLeNet with the recommended weights argument
model = models.googlenet(weights=GoogLeNet_Weights.IMAGENET1K_V1)

# Change the final fully connected layer for 10 classes (CIFAR-10)
model.fc = nn.Linear(model.fc.in_features, 10)

In [5]:
for name, param in model.named_parameters():
    if name.startswith(('conv1', 'conv2', 'inception3a', 'inception3b')):
        param.requires_grad = False

In [6]:
#Data Augmentation for training
robust_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Random crop and resize
    transforms.RandomHorizontalFlip(),                    # Random horizontal flip
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Color jitter
    transforms.RandomRotation(15),                        # Random rotation
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),  # Random blur
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [7]:
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from sklearn.model_selection import train_test_split

# Dataset for original images
class CIFAR10TorchDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        label = self.labels[idx][0]
        img = img.astype('uint8')
        if self.transform:
            img = self.transform(img)
        return img, label

# Flatten y_train for stratification
y_train_flat = y_train.flatten()

# Split train into train/val (80/20) with stratification
x_train_split, x_val, y_train_split, y_val = train_test_split(
    x_train, y_train, test_size=0.2, random_state=42, stratify=y_train_flat
)

# --- Augmentation setup ---
robust_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomRotation(15),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Datasets: both normal and robust for train, only normal for val/test
train_dataset_normal = CIFAR10TorchDataset(x_train_split, y_train_split, transform=transform)
train_dataset_robust = CIFAR10TorchDataset(x_train_split, y_train_split, transform=robust_transform)
from torch.utils.data import ConcatDataset
train_dataset = ConcatDataset([train_dataset_normal, train_dataset_robust])

val_dataset = CIFAR10TorchDataset(x_val, y_val, transform=transform)
test_dataset = CIFAR10TorchDataset(x_test, y_test, transform=transform)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [8]:
from tqdm import tqdm

def train(model, train_loader, val_loader, device, epochs=5, lr=1e-3, patience=5):
    import torch
    import torch.nn as nn
    import torch.optim as optim

    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]", leave=False)
        for images, labels in loop:
            images = images.to(device)
            labels = labels.to(device).squeeze()
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)
            loop.set_postfix(loss=loss.item())
        train_loss = running_loss / total
        train_acc = correct / total

        # Validation
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        loop_val = tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} [Val]", leave=False)
        with torch.no_grad():
            for images, labels in loop_val:
                images = images.to(device)
                labels = labels.to(device).squeeze()
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)
                _, predicted = outputs.max(1)
                val_correct += predicted.eq(labels).sum().item()
                val_total += labels.size(0)
                loop_val.set_postfix(loss=loss.item())
        val_loss /= val_total
        val_acc = val_correct / val_total

        print(f"Epoch {epoch+1}/{epochs} | "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

       # Path per salvare i pesi del modello
        best_model_path = "best_model_googleLenet_aug.pth"

        # Early stopping check
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), best_model_path)  # Save best model to disk
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                model.load_state_dict(torch.load(best_model_path))  # Restore best model from disk
                break

In [9]:
print(torch.cuda.is_available())  # Should be True
print(torch.cuda.device_count())  # Should be > 0
print(torch.cuda.get_device_name(0))  # Should return GPU name

True
1
NVIDIA GeForce RTX 4090


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
print(f"Using device: {device}")
num_epochs = 100
learning_rate = 1e-4

train(model, train_loader, val_loader, device, epochs=num_epochs, lr=learning_rate, patience=5)

Using device: cuda


Epoch 1/100 | Train Loss: 0.4030, Train Acc: 0.8778 | Val Loss: 0.1714, Val Acc: 0.9441


Epoch 2/100 | Train Loss: 0.1351, Train Acc: 0.9566 | Val Loss: 0.1458, Val Acc: 0.9518


Epoch 3/100 | Train Loss: 0.0868, Train Acc: 0.9721 | Val Loss: 0.1484, Val Acc: 0.9528


Epoch 4/100 | Train Loss: 0.0660, Train Acc: 0.9786 | Val Loss: 0.1460, Val Acc: 0.9542


Epoch 5/100 | Train Loss: 0.0541, Train Acc: 0.9826 | Val Loss: 0.1485, Val Acc: 0.9527


Epoch 6/100 | Train Loss: 0.0464, Train Acc: 0.9849 | Val Loss: 0.1514, Val Acc: 0.9539


Epoch 7/100 | Train Loss: 0.0386, Train Acc: 0.9875 | Val Loss: 0.1576, Val Acc: 0.9555
Early stopping at epoch 7


C:\Users\manua\AppData\Local\Temp\ipykernel_6824\2965275938.py:73: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path))  # Restor

In [15]:
def convert_normalization(imgs):
    """
    Convert a batch of images from normalization (mean=0.5, std=0.5)
    to GoogLeNet normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]).
    imgs: torch.Tensor of shape (B, 3, H, W)
    Returns: torch.Tensor of same shape, normalized for GoogLeNet
    """
    # Unnormalize from (0.5, 0.5, 0.5) to [0, 1]
    imgs = imgs * 0.5 + 0.5
    # Normalize to GoogLeNet
    mean = torch.tensor([0.485, 0.456, 0.406], device=imgs.device).view(1, 3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225], device=imgs.device).view(1, 3, 1, 1)
    imgs = (imgs - mean) / std
    return imgs

In [17]:
def get_googlenet(pretrained=True):
    if pretrained:
        model = models.googlenet(weights=GoogLeNet_Weights.IMAGENET1K_V1)
    else:
        model = models.googlenet(weights=None, init_weights=True)  # Explicit init
    model.fc = nn.Linear(model.fc.in_features, 10)
    return model

In [21]:
model_path = "best_model_googleLenet_aug.pth"
model = get_googlenet(pretrained=True)
model.to(device)
model.load_state_dict(torch.load(model_path, map_location=device, weights_only=True))
model.eval()
correct_test = 0
total_test = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

test_accuracy = 100 * correct_test / total_test
print(f"Test Accuracy: {test_accuracy:.2f}%")

Test Accuracy: 95.40%


In [22]:
import pandas as pd
from skimage.metrics import peak_signal_noise_ratio as psnr

def run_attacks_metrics(model, test_loader, device, epsilons=[0.01, 0.03, 0.05]):
    import foolbox as fb

    model.eval()
    fmodel = fb.PyTorchModel(model, bounds=(-1, 1))
    class_names = ('airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    results = []

    # For aggregate confusion
    fgsm_agg_conf = np.zeros((10, 10), dtype=int)
    pgd_agg_conf = np.zeros((10, 10), dtype=int)

    # Collect all test images and labels
    all_images = []
    all_labels = []
    for images, labels in test_loader:
        all_images.append(images)
        all_labels.append(labels)
    all_images = torch.cat(all_images, dim=0)
    all_labels = torch.cat(all_labels, dim=0)

    total_images = all_images.shape[0]

    fgsm_conf_matrices = []
    pgd_conf_matrices = []

    # PSNR metrics
    fgsm_psnr_per_eps = {}
    fgsm_psnr_per_class = {}
    pgd_psnr_per_eps = {}
    pgd_psnr_per_class = {}

    for eps in epsilons:
        batch_size = 128
        clean_correct = 0
        fgsm_correct = 0
        pgd_correct = 0
        total = 0

        fgsm_confusion = np.zeros((10, 10), dtype=int)
        pgd_confusion = np.zeros((10, 10), dtype=int)

        # For PSNR
        fgsm_psnr_list = []
        fgsm_psnr_per_class_list = [[] for _ in range(10)]
        pgd_psnr_list = []
        pgd_psnr_per_class_list = [[] for _ in range(10)]

        for i in range(0, total_images, batch_size):
            batch_imgs = all_images[i:i+batch_size].to(device)
            batch_lbls = all_labels[i:i+batch_size].to(device)

            attack_fgsm = fb.attacks.FGSM()
            advs_fgsm, _, _ = attack_fgsm(fmodel, batch_imgs, batch_lbls, epsilons=eps)
            attack_pgd = fb.attacks.LinfPGD(steps=10, rel_stepsize=0.1)
            advs_pgd, _, _ = attack_pgd(fmodel, batch_imgs, batch_lbls, epsilons=eps)

            batch_imgs_norm = convert_normalization(batch_imgs)  # Convert images for model
            advs_fgsm_norm = convert_normalization(advs_fgsm)  # Convert FGSM adversarial images
            advs_pgd_norm = convert_normalization(advs_pgd)  # Convert PGD adversarial images

            clean_pred = model(batch_imgs_norm).argmax(axis=1)
            fgsm_pred = model(advs_fgsm_norm).argmax(axis=1)
            pgd_pred = model(advs_pgd_norm).argmax(axis=1)

            clean_correct += (clean_pred == batch_lbls).sum().item()
            fgsm_correct += (fgsm_pred == batch_lbls).sum().item()
            pgd_correct += (pgd_pred == batch_lbls).sum().item()
            total += batch_lbls.size(0)

            for t, p in zip(batch_lbls.cpu().numpy(), fgsm_pred.cpu().numpy()):
                fgsm_confusion[t, p] += 1
                fgsm_agg_conf[t, p] += 1
            for t, p in zip(batch_lbls.cpu().numpy(), pgd_pred.cpu().numpy()):
                pgd_confusion[t, p] += 1
                pgd_agg_conf[t, p] += 1

            # FGSM PSNR calculation (unnormalize to [0,1] for PSNR)
            batch_imgs_unnorm = (batch_imgs_norm * torch.tensor([0.229, 0.224, 0.225], device=device).view(1,3,1,1)) + torch.tensor([0.485, 0.456, 0.406], device=device).view(1,3,1,1)
            advs_fgsm_unnorm = (advs_fgsm_norm * torch.tensor([0.229, 0.224, 0.225], device=device).view(1,3,1,1)) + torch.tensor([0.485, 0.456, 0.406], device=device).view(1,3,1,1)
            batch_imgs_unnorm = torch.clamp(batch_imgs_unnorm, 0, 1)
            advs_fgsm_unnorm = torch.clamp(advs_fgsm_unnorm, 0, 1)
            for j in range(batch_imgs_unnorm.shape[0]):
                psnr_fgsm = psnr(
                    batch_imgs_unnorm[j].cpu().numpy(),
                    advs_fgsm_unnorm[j].cpu().numpy(),
                    data_range=1.0
                )
                fgsm_psnr_list.append(psnr_fgsm)
                label = int(batch_lbls[j].item())
                fgsm_psnr_per_class_list[label].append(psnr_fgsm)

            # PGD PSNR calculation (unnormalize to [0,1] for PSNR)
            advs_pgd_unnorm = (advs_pgd_norm * torch.tensor([0.229, 0.224, 0.225], device=device).view(1,3,1,1)) + torch.tensor([0.485, 0.456, 0.406], device=device).view(1,3,1,1)
            advs_pgd_unnorm = torch.clamp(advs_pgd_unnorm, 0, 1)
            for j in range(batch_imgs_unnorm.shape[0]):
                psnr_pgd = psnr(
                    batch_imgs_unnorm[j].cpu().numpy(),
                    advs_pgd_unnorm[j].cpu().numpy(),
                    data_range=1.0
                )
                pgd_psnr_list.append(psnr_pgd)
                label = int(batch_lbls[j].item())
                pgd_psnr_per_class_list[label].append(psnr_pgd)

        clean_acc = 100 * clean_correct / total
        fgsm_acc = 100 * fgsm_correct / total
        pgd_acc = 100 * pgd_correct / total
        results.append({'epsilon': eps, 'clean_acc': clean_acc, 'fgsm_acc': fgsm_acc, 'pgd_acc': pgd_acc})

        # Store PSNR metrics
        fgsm_psnr_per_eps[eps] = np.mean(fgsm_psnr_list) if fgsm_psnr_list else float('nan')
        fgsm_psnr_per_class[eps] = [np.mean(fgsm_psnr_per_class_list[c]) if fgsm_psnr_per_class_list[c] else float('nan') for c in range(10)]
        pgd_psnr_per_eps[eps] = np.mean(pgd_psnr_list) if pgd_psnr_list else float('nan')
        pgd_psnr_per_class[eps] = [np.mean(pgd_psnr_per_class_list[c]) if pgd_psnr_per_class_list[c] else float('nan') for c in range(10)]

        fgsm_conf_matrices.append(fgsm_confusion.copy())
        pgd_conf_matrices.append(pgd_confusion.copy())

    df_results = pd.DataFrame(results)
    return {
        "fgsm_conf_matrices": fgsm_conf_matrices,
        "pgd_conf_matrices": pgd_conf_matrices,
        "fgsm_agg_conf": fgsm_agg_conf,
        "pgd_agg_conf": pgd_agg_conf,
        "class_names": class_names,
        "results": df_results,
        "fgsm_psnr_per_eps": fgsm_psnr_per_eps,
        "fgsm_psnr_per_class": fgsm_psnr_per_class,
        "pgd_psnr_per_eps": pgd_psnr_per_eps,
        "pgd_psnr_per_class": pgd_psnr_per_class
    }

In [23]:
def print_attack_metrics(metrics, attack_type="pgd", save_prefix=None):
    """
    Print and optionally save aggregate confusion, per-class confusion, and accuracy table for FGSM or PGD attacks.
    attack_type: "fgsm" or "pgd"
    save_prefix: if provided, saves CSVs with this prefix (e.g., "student_pgd")
    """
    import pandas as pd
    import numpy as np

    assert attack_type in ("fgsm", "pgd"), "attack_type must be 'fgsm' or 'pgd'"

    agg_conf_key = f"{attack_type}_agg_conf"
    psnr_per_eps_key = f"{attack_type}_psnr_per_eps"
    psnr_per_class_key = f"{attack_type}_psnr_per_class"

    print(f"\nAggregate {attack_type.upper()} confusion (all epsilons):")
    agg_conf = metrics[agg_conf_key]
    agg_df = pd.DataFrame(agg_conf, index=metrics["class_names"], columns=metrics["class_names"])
    print(agg_df)
    if save_prefix:
        agg_df.to_csv(f"{save_prefix}_agg_conf_{attack_type}.csv")

    # Calculate mean PSNR per epsilon and per class
    if psnr_per_eps_key in metrics and psnr_per_class_key in metrics:
        mean_psnr_per_eps = metrics[psnr_per_eps_key]  # dict: epsilon -> mean psnr
        mean_psnr_per_class = metrics[psnr_per_class_key]  # dict: epsilon -> [mean psnr per class]
    else:
        print(f"Warning: {attack_type.upper()} PSNR metrics not found in metrics dict, computing for last epsilon only.")
        mean_psnr_per_eps = {}
        mean_psnr_per_class = {}

    # Per-class confusion summary with mean PSNR per class
    summary = []
    for idx, row in enumerate(agg_conf):
        true_label = metrics["class_names"][idx]
        row_copy = row.copy()
        row_copy[idx] = 0
        total_confused = row_copy.sum()
        if total_confused == 0:
            most_confused = "-"
            count = 0
            percentage = 0.0
        else:
            most_confused_idx = np.argmax(row_copy)
            most_confused = metrics["class_names"][most_confused_idx]
            count = row_copy[most_confused_idx]
            percentage = 100.0 * count / total_confused
        # Get mean PSNR for this class (for the last epsilon)
        mean_psnr = None
        if psnr_per_class_key in metrics and metrics[psnr_per_class_key]:
            last_eps = list(metrics[psnr_per_class_key].keys())[-1]
            mean_psnr = metrics[psnr_per_class_key][last_eps][idx]
        summary.append({
            "True Label": true_label,
            "Most Confused With": most_confused,
            "Count": count,
            "Percentage": f"{percentage:.2f}%",
            "Mean PSNR": f"{mean_psnr:.2f}" if mean_psnr is not None else "-"
        })
    summary_df = pd.DataFrame(summary)
    print(summary_df.to_markdown(index=False))
    if save_prefix:
        summary_df.to_csv(f"{save_prefix}_perclass_{attack_type}.csv", index=False)

    # Show accuracy table with mean PSNR per epsilon
    print(f"\nAccuracy Table ({attack_type.upper()}):")
    df_results = metrics["results"]
    if psnr_per_eps_key in metrics:
        df_results = df_results.copy()
        col_name = f"mean_psnr_{attack_type}"
        df_results[col_name] = df_results["epsilon"].map(lambda eps: f"{metrics[psnr_per_eps_key][eps]:.2f}")
    print(df_results.to_markdown(index=False))
    if save_prefix:
        df_results.to_csv(f"{save_prefix}_accuracy_{attack_type}.csv", index=False)

In [24]:
import torch.nn as nn
from torchvision.models import GoogLeNet_Weights
import foolbox as fb
# Foolbox setup
transform_fgsm = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_dataset_attack = CIFAR10TorchDataset(x_test, y_test, transform=transform_fgsm)
test_loader_attack = DataLoader(test_dataset_attack, batch_size=128, shuffle=False)

In [25]:
epsilons = np.arange(0.05, 0.21, 0.05)
metrics_student = run_attacks_metrics(model, test_loader_attack, device, epsilons=epsilons)

In [26]:
print_attack_metrics(metrics_student, attack_type="fgsm")


Aggregate FGSM confusion (all epsilons):
            airplane  automobile  bird   cat  deer  dog  frog  horse  ship  \
airplane         369          21    83  3278    24    0    24      0   186   
automobile        10         781     2  3000     4    0    24      0    48   
bird              40           0   295  3461    35   14   142      4     8   
cat                5           2    18  3824    11   29   106      1     1   
deer               5           0    44  3485   283    8   158      7     9   
dog                1           0    34  3667    26  176    80     12     4   
frog               3           2    22  3343    45    8   569      1     7   
horse             12           0    32  3479   203   17    35    212     9   
ship              33          31    23  3145     5    6    40      1   702   
truck             20         151     3  3208     9    3     8      3    58   

            truck  
airplane       15  
automobile    131  
bird            1  
cat             3  

In [27]:
print_attack_metrics(metrics_student, attack_type="pgd")


Aggregate PGD confusion (all epsilons):
            airplane  automobile  bird   cat  deer   dog  frog  horse  ship  \
airplane           0          46   744  2129   180    55   411      9   379   
automobile       153           0   171  2349    36    73   269      8   152   
bird             197           8     0  3051   161   215   333     20    11   
cat               76          63   304     3   274  1240  1864    119    20   
deer              62           4   607  2656     0   171   400     87    10   
dog               45           2   506  2731   113     0   543     50     9   
frog              31           2   347  3372   115   111     0      8    10   
horse             34           1   191  2554   586   307   320      0     3   
ship             240          99   307  2774    86    44   394      1     0   
truck            107         408   197  2750    64    45   303     24   102   

            truck  
airplane       47  
automobile    789  
bird            4  
cat      